## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Diving into Delta Lake

<!-- You can run this notebook in a Databricks environment. Specifically, this notebook has been designed to run in [Databricks Community Edition](http://community.cloud.databricks.com/) as well. -->
To run this notebook, you have to [create a cluster](https://docs.databricks.com/clusters/create.html) with version **Databricks Runtime 7.4 or later** and [attach this notebook](https://docs.databricks.com/notebooks/notebooks-manage.html#attach-a-notebook-to-a-cluster) to that cluster. <br/>

### Source Data for this notebook
The data used is a modified version of the public data from [Lending Club](https://www.kaggle.com/wendykan/lending-club-loan-data). It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

[link to tutorial](https://databricks.com/discover/demos/delta-lake)

In [0]:
db = "deltadb"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[2]: DataFrame[key: string, value: string]

In [0]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *


def my_checkpoint_dir(): 
  return "/tmp/delta_demo/chkpt/%s" % str(random.randint(0, 10000))

# User-defined function to generate random state
@udf(returnType=StringType())
def random_state():
  return str(random.choice(["CA", "TX", "NY", "WA"]))


# Function to start a streaming query with a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream(table_format, table_name, schema_ok=False, type="batch"):
  
  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 500).load()
    .withColumn("loan_id", 10000 + col("value"))
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer"))
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000))
    .withColumn("addr_state", random_state())
    .withColumn("type", lit(type)))
    
  if schema_ok:
    stream_data = stream_data.select("loan_id", "funded_amnt", "paid_amnt", "addr_state", "type", "timestamp")
      
  query = (stream_data.writeStream
    .format(table_format)
    .option("checkpointLocation", my_checkpoint_dir())
    .trigger(processingTime = "5 seconds")
    .table(table_name))

  return query

In [0]:
# Function to stop all streaming queries 
def stop_all_streams():
    print("Stopping all streams")
    for s in spark.streams.active:
        try:
            s.stop()
        except:
            pass
    print("Stopped all streams")
    dbutils.fs.rm("/tmp/delta_demo/chkpt/", True)


def cleanup_paths_and_tables():
    dbutils.fs.rm("/tmp/delta_demo/", True)
    dbutils.fs.rm("file:/dbfs/tmp/delta_demo/loans_parquet/", True)
        
    for table in ["deltadb.loans_parquet", "deltadb.loans_delta", "deltadb.loans_delta2"]:
        spark.sql(f"DROP TABLE IF EXISTS {table}")
    
cleanup_paths_and_tables()

In [0]:
%sh mkdir -p /dbfs/tmp/delta_demo/loans_parquet/; wget -O /dbfs/tmp/delta_demo/loans_parquet/loans.parquet https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet

--2021-08-25 20:39:23-- https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.70.206, 104.17.74.206, 104.17.72.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.70.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’

 0K .......... .......... .......... .......... .......... 31% 3.93M 0s
 50K .......... .......... .......... .......... .......... 62% 7.72M 0s
 100K .......... .......... .......... .......... .......... 93% 11.3M 0s
 150K .......... 100% 9.66M=0.02s

2021-08-25 20:39:23 (6.50 MB/s) - ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’ saved [164631/164631]

# Getting started with <img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>

An open-source storage layer for data lakes that brings ACID transactions to Apache Spark™ and big data workloads.

* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box. 
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Open Format**: Stored as Parquet format in blob storage.
* **Audit History**: History of all the operations that happened in the table.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Convert to Delta Lake format

Delta Lake is 100% compatible with Apache Spark&trade;, which makes it easy to get started with if you already use Spark for your big data workflows.
Delta Lake features APIs for **SQL**, **Python**, and **Scala**, so that you can use it in whatever language you feel most comfortable in.

<img src="https://databricks.com/wp-content/uploads/2020/12/simplysaydelta.png" width=600/>

In **Python**: Read your data into a Spark DataFrame, then write it out in Delta Lake format directly, with no upfront schema definition needed.

In [0]:
parquet_path = "file:/dbfs/tmp/delta_demo/loans_parquet/"

df = (spark.read.format("parquet").load(parquet_path)
      .withColumn("type", lit("batch"))
      .withColumn("timestamp", current_timestamp()))

df.write.format("delta").mode("overwrite").saveAsTable("loans_delta3")

**SQL:** Use `CREATE TABLE` statement with SQL (no upfront schema definition needed)

In [0]:
%sql
CREATE TABLE loans_delta4
USING delta
AS SELECT * FROM parquet.`/tmp/delta_demo/loans_parquet`

num_affected_rows,num_inserted_rows


**SQL**: Use `CONVERT TO DELTA` to convert Parquet files to Delta Lake format in place

In [0]:
%sql CONVERT TO DELTA parquet.`/tmp/delta_demo/loans_parquet`

### View the data in the Delta Lake table
**How many records are there, and what does the data look like?**

In [0]:
spark.sql("select count(*) from loans_delta3").show()
spark.sql("select * from loans_delta3").show(3)

+--------+
count(1)|
+--------+
 14705|
+--------+

+-------+-----------+---------+----------+-----+--------------------+
loan_id|funded_amnt|paid_amnt|addr_state| type| timestamp|
+-------+-----------+---------+----------+-----+--------------------+
 0| 1000| 182.22| CA|batch|2021-08-25 20:39:...|
 1| 1000| 361.19| WA|batch|2021-08-25 20:39:...|
 2| 1000| 176.26| TX|batch|2021-08-25 20:39:...|
+-------+-----------+---------+----------+-----+--------------------+
only showing top 3 rows

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified batch + streaming data processing with multiple concurrent readers and writers

### Write 2 different data streams into our Delta Lake table at the same time.

In [0]:
# Set up 2 streaming writes to our table
stream_query_A = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream A')
stream_query_B = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream B')

### Create 2 continuous streaming readers of our Delta Lake table to illustrate streaming progress.

In [0]:
# Streaming read #1
display(spark.readStream.format("delta").table("loans_delta").groupBy("type").count().orderBy("type"))

type,count
batch,14705
stream A,39000
stream B,36000


In [0]:
# Streaming read #2
display(spark.readStream.format("delta").table("loans_delta").groupBy("type", window("timestamp", "10 seconds")).count().orderBy("window"))

type,window,count
batch,"List(2021-08-25T19:42:30.000+0000, 2021-08-25T19:42:40.000+0000)",14705
stream A,"List(2021-08-25T19:43:50.000+0000, 2021-08-25T19:44:00.000+0000)",544
stream B,"List(2021-08-25T19:43:50.000+0000, 2021-08-25T19:44:00.000+0000)",190
stream B,"List(2021-08-25T19:44:00.000+0000, 2021-08-25T19:44:10.000+0000)",5000
stream A,"List(2021-08-25T19:44:00.000+0000, 2021-08-25T19:44:10.000+0000)",5000
stream A,"List(2021-08-25T19:44:10.000+0000, 2021-08-25T19:44:20.000+0000)",5000
stream B,"List(2021-08-25T19:44:10.000+0000, 2021-08-25T19:44:20.000+0000)",5000
stream B,"List(2021-08-25T19:44:20.000+0000, 2021-08-25T19:44:30.000+0000)",5000
stream A,"List(2021-08-25T19:44:20.000+0000, 2021-08-25T19:44:30.000+0000)",5000
stream B,"List(2021-08-25T19:44:30.000+0000, 2021-08-25T19:44:40.000+0000)",5000


### Add a batch query, just for good measure

In [0]:
%sql
SELECT addr_state, COUNT(*)
FROM loans_delta
GROUP BY addr_state

addr_state,count(1)
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


In [0]:
dbutils.notebook.exit("stop")

stop

In [0]:
stop_all_streams()

Stopping all streams
Stopped all streams

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) ACID Transactions

View the Delta Lake transaction log

In [0]:
%sql DESCRIBE HISTORY loans_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
12,2021-08-25T20:41:07.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 2f729de4-e630-4477-aa7d-0847738b1e04, epochId -> 5)",null,List(4213987394429508),0708-215551-forts756,10,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62860, numAddedFiles -> 1)",null
11,2021-08-25T20:41:05.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 8b5e9a87-f6ba-4eb7-8626-bd4de9e0e751, epochId -> 5)",null,List(4213987394429508),0708-215551-forts756,10,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62998, numAddedFiles -> 1)",null
10,2021-08-25T20:41:01.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 2f729de4-e630-4477-aa7d-0847738b1e04, epochId -> 4)",null,List(4213987394429508),0708-215551-forts756,8,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62043, numAddedFiles -> 1)",null
9,2021-08-25T20:41:00.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 8b5e9a87-f6ba-4eb7-8626-bd4de9e0e751, epochId -> 4)",null,List(4213987394429508),0708-215551-forts756,8,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 61654, numAddedFiles -> 1)",null
8,2021-08-25T20:40:57.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 8b5e9a87-f6ba-4eb7-8626-bd4de9e0e751, epochId -> 3)",null,List(4213987394429508),0708-215551-forts756,6,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62378, numAddedFiles -> 1)",null
7,2021-08-25T20:40:56.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 2f729de4-e630-4477-aa7d-0847738b1e04, epochId -> 3)",null,List(4213987394429508),0708-215551-forts756,6,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62321, numAddedFiles -> 1)",null
6,2021-08-25T20:40:52.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 2f729de4-e630-4477-aa7d-0847738b1e04, epochId -> 2)",null,List(4213987394429508),0708-215551-forts756,4,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62150, numAddedFiles -> 1)",null
5,2021-08-25T20:40:50.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 8b5e9a87-f6ba-4eb7-8626-bd4de9e0e751, epochId -> 2)",null,List(4213987394429508),0708-215551-forts756,4,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 63151, numAddedFiles -> 1)",null
4,2021-08-25T20:40:47.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 2f729de4-e630-4477-aa7d-0847738b1e04, epochId -> 1)",null,List(4213987394429508),0708-215551-forts756,2,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2000, numOutputBytes -> 49327, numAddedFiles -> 1)",null
3,2021-08-25T20:40:46.000+0000,4002569631682713,anpadill@microsoft.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 8b5e9a87-f6ba-4eb7-8626-bd4de9e0e751, epochId -> 1)",null,List(4213987394429508),0708-215551-forts756,2,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 63135, numAddedFiles -> 1)",null


<img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=1012/>

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use Schema Enforcement to protect data quality

To show you how schema enforcement works, let's create a new table that has an extra column -- `credit_score` -- that doesn't match our existing Delta Lake table schema.

#### Write DataFrame with extra column, `credit_score`, to Delta Lake table

In [0]:
# Generate `new_data` with additional column
new_column = [StructField("credit_score", IntegerType(), True)]
new_schema = StructType(spark.table("loans_delta").schema.fields + new_column)
data = [(99997, 10000, 1338.55, "CA", "batch", datetime.now(), 649),
        (99998, 20000, 1442.55, "NY", "batch", datetime.now(), 702)]

new_data = spark.createDataFrame(data, new_schema)
new_data.printSchema()

root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)
-- credit_score: integer (nullable = true)

In [0]:
# Uncommenting this cell will lead to an error because the schemas don't match.
# Attempt to write data with new column to Delta Lake table
new_data.write.format("delta").mode("append").saveAsTable("loans_delta")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-4213987394429545> in <module> 
 1 # Uncommenting this cell will lead to an error because the schemas don't match. 
 2 # Attempt to write data with new column to Delta Lake table 
 ----> 3 new_data . write . format ( "delta" ) . mode ( "append" ) . saveAsTable ( "loans_delta" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable (self, name, format, mode, partitionBy, **options) 
 1183 if format is not None : 
 1184 self . format ( format ) 
 -> 1185 self . _jwrite . saveAsTable ( name ) 
 1186 
 1187 def json(self, path, mode=None, compression=None, dateFormat=None, timestampFormat=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: bfcf2758-8711-420c-b5eb-9309e19eb479).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)


Data schema:
root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)
-- credit_score: integer (nullable = true)

**Schema enforcement helps keep our tables clean and tidy so that we can trust the data we have stored in Delta Lake.** The writes above were blocked because the schema of the new data did not match the schema of table (see the exception details). See more information about how it works [here](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html).

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use Schema Evolution to add new columns to schema

If we *want* to update our Delta Lake table to match this data source's schema, we can do so using schema evolution. Simply add the following to the Spark write command: `.option("mergeSchema", "true")`

In [0]:
new_data.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("loans_delta")

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (99997, 99998)

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score
99997,10000,1338.55,CA,batch,2021-08-25T19:46:15.319+0000,649
99998,20000,1442.55,NY,batch,2021-08-25T19:46:15.319+0000,702


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake Time Travel

Delta Lake’s time travel capabilities simplify building data pipelines for use cases including:

* Auditing Data Changes
* Reproducing experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

<img src="https://github.com/risan4841/img/blob/master/transactionallogs.png?raw=true" width=250/>

You can query snapshots of your tables by:
1. **Version number**, or
2. **Timestamp.**

using Python, Scala, and/or SQL syntax; for these examples we will use the SQL syntax.  

For more information, refer to the [docs](https://docs.delta.io/latest/delta-utility.html#history), or [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

#### Review Delta Lake Table History for  Auditing & Governance
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [0]:
%sql
DESCRIBE HISTORY loans_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
111,2021-01-04T20:23:00.000+0000,101001,Brenner.Heintz@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(8163273),0318-151752-abed99,110,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 1933, numOutputRows -> 2)",null
110,2021-01-04T20:20:57.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1e76a9c2-f557-46e3-865a-d0c6e6617d29, epochId -> 54)",null,List(8163273),0318-151752-abed99,108,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62263, numAddedFiles -> 1)",null
109,2021-01-04T20:20:56.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 67316880-2d22-417e-8103-577b47ecf139, epochId -> 54)",null,List(8163273),0318-151752-abed99,108,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62259, numAddedFiles -> 1)",null
108,2021-01-04T20:20:51.001+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1e76a9c2-f557-46e3-865a-d0c6e6617d29, epochId -> 53)",null,List(8163273),0318-151752-abed99,106,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 62463, numAddedFiles -> 1)",null
107,2021-01-04T20:20:51.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 67316880-2d22-417e-8103-577b47ecf139, epochId -> 53)",null,List(8163273),0318-151752-abed99,106,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 61777, numAddedFiles -> 1)",null
106,2021-01-04T20:20:47.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 67316880-2d22-417e-8103-577b47ecf139, epochId -> 52)",null,List(8163273),0318-151752-abed99,104,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2000, numOutputBytes -> 49651, numAddedFiles -> 1)",null
105,2021-01-04T20:20:46.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1e76a9c2-f557-46e3-865a-d0c6e6617d29, epochId -> 52)",null,List(8163273),0318-151752-abed99,104,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2000, numOutputBytes -> 50085, numAddedFiles -> 1)",null
104,2021-01-04T20:20:43.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1e76a9c2-f557-46e3-865a-d0c6e6617d29, epochId -> 51)",null,List(8163273),0318-151752-abed99,102,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1000, numOutputBytes -> 24890, numAddedFiles -> 1)",null
103,2021-01-04T20:20:42.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 67316880-2d22-417e-8103-577b47ecf139, epochId -> 51)",null,List(8163273),0318-151752-abed99,102,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1500, numOutputBytes -> 37950, numAddedFiles -> 1)",null
102,2021-01-04T20:20:40.000+0000,101001,Brenner.Heintz@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 1e76a9c2-f557-46e3-865a-d0c6e6617d29, epochId -> 50)",null,List(8163273),0318-151752-abed99,100,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2000, numOutputBytes -> 49643, numAddedFiles -> 1)",null


#### Use time travel to select and view the original version of our table (Version 0).
As you can see, this version contains the original 14,705 records in it.

In [0]:
spark.sql("SELECT * FROM loans_delta VERSION AS OF 0").show(3)
spark.sql("SELECT COUNT(*) FROM loans_delta VERSION AS OF 0").show()

+-------+-----------+---------+----------+-----+--------------------+
loan_id|funded_amnt|paid_amnt|addr_state| type| timestamp|
+-------+-----------+---------+----------+-----+--------------------+
 0| 1000| 182.22| CA|batch|2021-01-04 20:15:...|
 1| 1000| 361.19| WA|batch|2021-01-04 20:15:...|
 2| 1000| 176.26| TX|batch|2021-01-04 20:15:...|
+-------+-----------+---------+----------+-----+--------------------+
only showing top 3 rows

+--------+
count(1)|
+--------+
 14705|
+--------+

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

count(1)
319707


#### Rollback a table to a specific version using `RESTORE`

In [0]:
%sql RESTORE loans_delta VERSION AS OF 0

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

count(1)
14705


##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support: `DELETE`, `UPDATE`, `MERGE INTO`

Delta Lake brings ACID transactions and full DML support to data lakes.

>Parquet does **not** support these commands - they are unique to Delta Lake.

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `DELETE`: Handle GDPR or CCPA Requests on your Data Lake

Imagine that we are responding to a GDPR data deletion request. The user with loan ID #4420 wants us to delete their data. Here's how easy it is.

**View the user's data**

In [0]:
%sql
SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp
4420,22000,1050.94,TX,batch,2021-01-04T20:15:28.381+0000


**Delete the individual user's data with a single `DELETE` command using Delta Lake.**

Note: The `DELETE` command isn't supported in Parquet.

In [0]:
%sql
DELETE FROM loans_delta WHERE loan_id=4420;
-- Confirm the user's data was deleted
SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use time travel and `INSERT INTO` to add the user back into our table

In [0]:
%sql
INSERT INTO loans_delta
SELECT * FROM loans_delta VERSION AS OF 0
WHERE loan_id=4420

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp
4420,22000,1050.94,TX,batch,2021-01-04T20:15:28.381+0000


### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `UPDATE`: Modify the existing records in a table in one command

In [0]:
%sql UPDATE loans_delta SET funded_amnt = 22000 WHERE loan_id = 4420

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id = 4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp
4420,22000,1050.94,TX,batch,2021-01-04T20:15:28.381+0000


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Support Change Data Capture Workflows & Other Ingest Use Cases via `MERGE INTO`

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

<img src="https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif" alt='Merge process' width=600/>


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [0]:
# Create merge table with 1 row update, 1 insertion
data = [(4420, 22000, 21500.00, "NY", "update", datetime.now()),  # record to update
        (99999, 10000, 1338.55, "CA", "insert", datetime.now())]  # record to insert
schema = spark.table("loans_delta").schema
spark.createDataFrame(data, schema).createOrReplaceTempView("merge_table")
spark.sql("SELECT * FROM merge_table").show()

+-------+-----------+---------+----------+------+--------------------+
loan_id|funded_amnt|paid_amnt|addr_state| type| timestamp|
+-------+-----------+---------+----------+------+--------------------+
 4420| 22000| 21500.0| NY|update|2021-01-04 20:23:...|
 99999| 10000| 1338.55| CA|insert|2021-01-04 20:23:...|
+-------+-----------+---------+----------+------+--------------------+

In [0]:
%sql
MERGE INTO loans_delta AS l
USING merge_table AS m
ON l.loan_id = m.loan_id
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (4420, 99999)

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp
4420,22000,21500.0,NY,update,2021-01-04T20:23:52.362+0000
99999,10000,1338.55,CA,insert,2021-01-04T20:23:52.362+0000


## ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) File compaction and performance optimizations = faster queries

### Vacuum

In [0]:
%sql
-- Vacuum deletes all files no longer needed by the current version of the table.
VACUUM loans_delta

path
dbfs:/user/hive/warehouse/deltadb.db/loans_delta


### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Cache table in memory (Databricks Delta Lake only)

In [0]:
%sql CACHE SELECT * FROM loans_delta

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Z-Order Optimize (Databricks Delta Lake only)

In [0]:
%sql OPTIMIZE loans_delta ZORDER BY addr_state

path,metrics
null,"List(1, 2, List(165253, 165253, 165253.0, 1, 165253), List(1652, 165309, 83480.0, 2, 166961), 0, List(minCubeSize(107374182400), List(0, 0), List(2, 166961), 0, List(2, 166961), 1, null), 1)"


In [0]:
cleanup_paths_and_tables()

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>